# Day 5

In [1]:
ex1 = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [19]:
#data
d = open('day05.txt').read()

In [2]:
def process_data(inp):
    clean = [b.strip().splitlines() for a,b in [x.split(":") for x in inp.split("\n\n")]]
    seeds = [int(i) for i in [x.split(" ") for x in clean[0]][0]]
    maps = []
    for i in range(1,len(clean)):
        submap = []
        for m in clean[i]:
            submap.append([int(i) for i in m.split(" ")])
        maps.append(submap)

    return seeds, maps


In [3]:
def mapper(seed_in, maps):
    for m in maps:
        map_min = m[1] # source
        map_max = m[1] + m[2] # source + range

        if seed_in <= map_max and seed_in >= map_min: #if it's within the source range
            diff = seed_in - map_min # calculate difference between source and destination
            return  m[0] + diff # destination plus offset
    return seed_in

In [4]:
def p1(inp):

    seeds, maps = process_data(inp)

    best_location = None

    for s in seeds: #for each seed 
        for current_map in maps: # map through
            s = mapper(s,current_map)
            
        if best_location is None or s < best_location:
            best_location = s
    return best_location

In [20]:
p1(ex1)

35

In [21]:
p1(d)

388071289

## Part 2

Going to need to do something different to avoid bruteforcing ~100,000,000+ seeds.

Can we process ranges of seeds rather than individual seeds? 

Use list of ranges [min_seed,max_seed +1]

In [9]:
#seeds, maps = process_data(ex1)
seeds, maps = process_data(open('day05.txt').read())


In [14]:
# turn seeds data into a list of seed_ranges like [min_seed, max_seed + 1]
def get_seed_ranges(seeds):
    seed_range = []
    for i in range(0,int(len(seeds)/2)):
        i = i * 2
        seed_range.append([seeds[i],seeds[i]+(seeds[i+1])])
    return seed_range

In [15]:
# check whether to ranges in the format [min, max+1], if they overlap, return the overlap range. else return None
def overlap_range(x, y):
    try:
        # Check for no overlap
        if max(x[0], y[0]) >= min(x[-1], y[-1]):
            return None
        
        if x[0] == x[1] or y[0] == y[1]:
            return None

        # Calculate overlap
        return [max(x[0], y[0]), min(x[-1], y[-1]) ]
    except Exception as e:
        print(f"An error occurred with inputs x: {x}, y: {y}. Error: {e}")
        raise  # Re-raise the exception


In [16]:
def mapper_range(seed_ranges, current_map):
# take in a seed range, output mapped ranges
    output = [] # list of output ranges

    for s in seed_ranges:
        process_seeds = [s] #seed range to be processed against each map
        
        for m in current_map: # need to iterate through each map

                range_m = [m[1],m[1]+m[2]] #the map represented as a range
                shift = m[0] - m[1] # shift amount

                carry_forward = [] # store ranges that need to be processed in the next map

                for sub_range in process_seeds: # we might have before and afters from a previous mapping to check
                    overlap = overlap_range(sub_range,range_m) # the bit of the source range that needs to be mapped

                    if overlap: #if there's an overlap, add the mapped range to the output
                        mapped_overlap = [min(overlap) + shift , max(overlap) + shift] # new mapped range
                        output.append(mapped_overlap) # this is definitely in our output
                        if sub_range[0] < min(overlap):
                            carry_forward.append([sub_range[0],min(overlap)]) # range before, still to be checked against other maps
                        if max(overlap) < max(sub_range):
                            carry_forward.append([max(overlap),max(sub_range)]) # range after, still to be checked against other maps
                    
                    else: #no overlaps, we need to carry forward the whole sub_range to the next map
                        carry_forward.append(sub_range)

                # these need to be processed by the next map
                process_seeds = carry_forward

        #at the end of all the mapping passes, any unprocessed contents of s just map as they are
        for element in process_seeds:
            output.append(element)

    return output

In [17]:
def p2(inp):
    seeds, maps = process_data(inp)
    seed_range = get_seed_ranges(seeds)

    for current_map in maps:
        seed_range = mapper_range(seed_range, current_map)
    
    return min(min(x) for x in seed_range if x)

In [23]:
p2(ex1)

46

In [22]:
# part 2 answer
p2(d)

84206669